# Data Preparation for Tableau

## 1. Data Loading

In [1]:
import pandas as pd
import numpy as np
import locale

csv_folder_path = "D:/airline_csvs/"

def set_display_locale(lang):
    if lang == "en":
        locale.setlocale(locale.LC_ALL, "en_US.UTF-8")
        def fmt(x):
            if abs(x) < 1000:
                return f"{x:,.2f}"
            else:
                return f"{x:,.0f}"
    elif lang == "de":
        locale.setlocale(locale.LC_ALL, "de_DE.UTF-8")
        def fmt(x):
            if abs(x) < 1000:
                return locale.format_string("%.2f", x, grouping=True)
            else:
                return locale.format_string("%.0f", x, grouping=True)
    else:
        raise ValueError("lang must be 'en' or 'de'")

    pd.options.display.float_format = fmt

set_display_locale("de")
#pd.reset_option("display.float_format")

In [2]:
airports = pd.read_csv(csv_folder_path + "airports.csv", sep=";", decimal=",")
aircraft = pd.read_csv(csv_folder_path + "aircraft.csv", sep=";", decimal=",")
routes = pd.read_csv(csv_folder_path + "routes.csv", sep=";", decimal=",")
frequent_flyer_discounts = pd.read_csv(csv_folder_path + "frequent_flyer_discounts.csv", sep=";", decimal=",")

In [3]:
flights_2023 = (
    pd.read_csv(csv_folder_path + "flights.csv", sep=";", decimal=",",
    parse_dates=["flight_date", "scheduled_departure", "scheduled_arrival", "actual_departure", "actual_arrival"])
    .query("flight_date.dt.year == 2023")
)

flights_booked_passengers_2023 = (
    pd.read_csv(csv_folder_path + "flights_booked_passengers.csv", sep=";", decimal=",",
    parse_dates=["flight_date", "scheduled_departure", "scheduled_arrival", "actual_departure", "actual_arrival"])
    .query("flight_date.dt.year == 2023")
)

flight_capacity_by_class_2023 = (
    pd.read_csv(csv_folder_path + "flight_capacity_by_class.csv", sep=";", decimal=",", parse_dates=["flight_date"])
    .query("flight_date.dt.year == 2023")
)

costs_per_flight_2023 = (
    pd.read_csv(csv_folder_path + "costs_per_flight.csv", sep=";", decimal=",", parse_dates=["flight_date"])
    .query("flight_date.dt.year == 2023")
)

flight_class_cost_shares_2023 = (
    pd.read_csv(csv_folder_path + "flight_class_cost_shares.csv", sep=";", decimal=",", parse_dates=["flight_date"])
    .query("flight_date.dt.year == 2023")
)

weather_2023 = (
    pd.read_csv(csv_folder_path + "weather.csv", sep=";", decimal=",", parse_dates=["observation_time"])
    .query("observation_time.dt.year == 2023")
)

In [4]:
bookings_2023 = (
    pd.read_csv(csv_folder_path + "bookings.csv", sep=";", decimal=",", parse_dates=["flight_date", "booking_time"])
    .query("flight_date.dt.year == 2023")
)

In [5]:
customer_id_mask = bookings_2023["customer_id"].unique().tolist()
customers_2023 = pd.read_csv(csv_folder_path + "customers_cleaned.csv", sep=";", decimal=",").query("customer_id in @customer_id_mask")

## 2. Lookup Tables

In [6]:
airport_names_clean = airports.copy()

airport_names_clean["airport_name"] = (
    airport_names_clean["airport_name"]
    .replace("São Paulo/Guarulhos–Governador André Franco Montoro International Airport", "São Paulo International Airport")
    .replace("Adolfo Suárez Madrid–Barajas Airport","Madrid–Barajas Airport")
    .replace("Beirut–Rafic Hariri International Airport","Beirut International Airport")
)

airport_names_clean

,airport_code,airport_name,city,country,coordinates,climate_region,hub_status
0,ALG,Houari Boumediene Airport,Algiers,Algeria,"(3.2154,36.691)",Mediterranean,none
1,AMM,Queen Alia International Airport,Amman,Jordan,"(35.9932,31.7226)",Desert,none
2,AMS,Amsterdam Schiphol Airport,Amsterdam,Netherlands,"(4.7683,52.3105)",Temperate,none
3,ARN,Stockholm Arlanda Airport,Stockholm,Sweden,"(17.9186,59.6519)",Cold,none
4,ATH,Athens International Airport,Athens,Greece,"(23.9475,37.9364)",Mediterranean,none
5,BCN,Barcelona–El Prat Airport,Barcelona,Spain,"(2.0833,41.2974)",Mediterranean,none
6,BER,Berlin Brandenburg Airport,Berlin,Germany,"(13.5033,52.3667)",Temperate,none
7,BEY,Beirut International Airport,Beirut,Lebanon,"(35.4884,33.8209)",Mediterranean,none
8,BRU,Brussels Airport,Brussels,Belgium,"(4.4844,50.9014)",Temperate,none
9,CAI,Cairo International Airport,Cairo,Egypt,"(31.4056,30.1219)",Desert,none


In [9]:
routes_named = (
    routes[["line_number", "departure_airport_code", "arrival_airport_code", "distance_km"]]
    .merge(airport_names_clean[["airport_code", "airport_name"]], left_on="departure_airport_code", right_on="airport_code")
    .rename(columns={"airport_name": "departure_airport_name"})
    .drop(columns=["airport_code"])
    .merge(airport_names_clean[["airport_code", "airport_name"]], left_on="arrival_airport_code", right_on="airport_code")
    .rename(columns={"airport_name": "arrival_airport_name"})
    .drop(columns=["airport_code"])
)

routes_named["route_name_short"] = routes_named["departure_airport_code"] + " → " + routes_named["arrival_airport_code"]
routes_named["route_name_long"] = routes_named["departure_airport_name"] + " → " + routes_named["arrival_airport_name"]

distance_conditions = (routes_named["distance_km"] < 1500, routes_named["distance_km"] < 5000, routes_named["distance_km"] >= 5000)
distance_choices = ("(1) Short-haul (< 1500 km)", "(2) Medium-haul (1500–4999 km)", "(3) Long-haul (≥ 5000 km)")
routes_named["distance_category"] = np.select(distance_conditions, distance_choices, default="Unknown")

routes_named = routes_named[["line_number", "route_name_short", "route_name_long", "distance_category"]]
routes_named

,line_number,route_name_short,route_name_long,distance_category
0,101,FRA → LHR,Frankfurt Airport → London Heathrow Airport,(1) Short-haul (< 1500 km)
1,102,FRA → BER,Frankfurt Airport → Berlin Brandenburg Airport,(1) Short-haul (< 1500 km)
2,103,FRA → HAM,Frankfurt Airport → Hamburg Airport,(1) Short-haul (< 1500 km)
3,104,FRA → VIE,Frankfurt Airport → Vienna International Airport,(1) Short-haul (< 1500 km)
4,105,FRA → MUC,Frankfurt Airport → Munich Airport,(1) Short-haul (< 1500 km)
...,...,...,...,...
137,238,ICN → FRA,Incheon International Airport → Frankfurt Airport,(3) Long-haul (≥ 5000 km)
138,239,GRU → FRA,São Paulo International Airport → Frankfurt Ai...,(3) Long-haul (≥ 5000 km)
139,240,NRT → FRA,Narita International Airport → Frankfurt Airport,(3) Long-haul (≥ 5000 km)
140,241,JNB → FRA,O. R. Tambo International Airport → Frankfurt ...,(3) Long-haul (≥ 5000 km)


In [10]:
travel_seasons = ["(1) Jan & Feb", "(2) Spring", "(3) Summer", "(4) Autumn", "(5) December"]
routes_seasons = pd.DataFrame()
for season in travel_seasons:
    df = routes_named.copy()
    df["travel_season"] = season
    routes_seasons = pd.concat([routes_seasons, df])

weekday_groups = ["(1) Mon/Wed/Thu", "(2) Tue/Sat", "(3) Fri/Sun"]
routes_lookup = pd.DataFrame()
for weekday_group in weekday_groups:
    df = routes_seasons.copy()
    df["weekday_group"] = weekday_group
    routes_lookup = pd.concat([routes_lookup, df])

routes_lookup = routes_lookup.reset_index(drop=True)
routes_lookup

,line_number,route_name_short,route_name_long,distance_category,travel_season,weekday_group
0,101,FRA → LHR,Frankfurt Airport → London Heathrow Airport,(1) Short-haul (< 1500 km),(1) Jan & Feb,(1) Mon/Wed/Thu
1,102,FRA → BER,Frankfurt Airport → Berlin Brandenburg Airport,(1) Short-haul (< 1500 km),(1) Jan & Feb,(1) Mon/Wed/Thu
2,103,FRA → HAM,Frankfurt Airport → Hamburg Airport,(1) Short-haul (< 1500 km),(1) Jan & Feb,(1) Mon/Wed/Thu
3,104,FRA → VIE,Frankfurt Airport → Vienna International Airport,(1) Short-haul (< 1500 km),(1) Jan & Feb,(1) Mon/Wed/Thu
4,105,FRA → MUC,Frankfurt Airport → Munich Airport,(1) Short-haul (< 1500 km),(1) Jan & Feb,(1) Mon/Wed/Thu
...,...,...,...,...,...,...
2125,238,ICN → FRA,Incheon International Airport → Frankfurt Airport,(3) Long-haul (≥ 5000 km),(5) December,(3) Fri/Sun
2126,239,GRU → FRA,São Paulo International Airport → Frankfurt Ai...,(3) Long-haul (≥ 5000 km),(5) December,(3) Fri/Sun
2127,240,NRT → FRA,Narita International Airport → Frankfurt Airport,(3) Long-haul (≥ 5000 km),(5) December,(3) Fri/Sun
2128,241,JNB → FRA,O. R. Tambo International Airport → Frankfurt ...,(3) Long-haul (≥ 5000 km),(5) December,(3) Fri/Sun


In [11]:
routes_lookup.to_csv(csv_folder_path + "routes_lookup.csv", index=False, sep=";", decimal=",")

In [ ]:
frequent_flyer_discounts_ordered = frequent_flyer_discounts.copy()

frequent_flyer_discounts_ordered["frequent_flyer_status"] = (
    frequent_flyer_discounts_ordered["frequent_flyer_status"]
    .replace("No Status", "(0) No Status")
    .replace("Basic", "(1) Basic")
    .replace("Silver", "(2) Silver")
    .replace("Gold", "(3) Gold")
    .replace("Platinum", "(4) Platinum")
)

frequent_flyer_discounts_ordered

,frequent_flyer_status_code,frequent_flyer_status,min_flights_yearly,frequent_flyer_discount
0,0-N,(0) No Status,NaN,"0,00"
1,1-B,(1) Basic,"4,00","0,01"
2,2-S,(2) Silver,"6,00","0,03"
3,3-G,(3) Gold,"8,00","0,08"
4,4-P,(4) Platinum,"11,00","0,13"


In [23]:
genders = customers_2023[pd.notna(customers_2023["gender"])]["gender"].unique().tolist()
customer_genders = pd.DataFrame()
for gender in genders:
    df = frequent_flyer_discounts_ordered.copy()
    df["gender"] = gender
    customer_genders = pd.concat([customer_genders, df])

nationalities = customers_2023[pd.notna(customers_2023["nationality"])]["nationality"].unique().tolist()
customer_nationalities = pd.DataFrame()
for nat in nationalities:
    df = customer_genders.copy()
    df["nationality"] = nat
    customer_nationalities = pd.concat([customer_nationalities, df])

age_groups = ["(1) Age <= 24", "(2) Age 25-34", "(3) Age 35-44", "(4) Age 45-60", "(5) Age > 60", "(6) Age unknown"]
customers_lookup = pd.DataFrame()
for age_group in age_groups:
    df = customer_nationalities.copy()
    df["age_group"] = age_group
    customers_lookup = pd.concat([customers_lookup, df])

customers_lookup

,frequent_flyer_status_code,frequent_flyer_status,min_flights_yearly,frequent_flyer_discount,gender,nationality,age_group
0,0-N,(0) No Status,NaN,"0,00",male,United States,(1) Age <= 24
1,1-B,(1) Basic,"4,00","0,01",male,United States,(1) Age <= 24
2,2-S,(2) Silver,"6,00","0,03",male,United States,(1) Age <= 24
3,3-G,(3) Gold,"8,00","0,08",male,United States,(1) Age <= 24
4,4-P,(4) Platinum,"11,00","0,13",male,United States,(1) Age <= 24
...,...,...,...,...,...,...,...
0,0-N,(0) No Status,NaN,"0,00",female,Australia,(6) Age unknown
1,1-B,(1) Basic,"4,00","0,01",female,Australia,(6) Age unknown
2,2-S,(2) Silver,"6,00","0,03",female,Australia,(6) Age unknown
3,3-G,(3) Gold,"8,00","0,08",female,Australia,(6) Age unknown


In [24]:
customers_lookup.to_csv(csv_folder_path + "customers_lookup.csv", index=False, sep=";", decimal=",")

## 3. Financial and Operational Performance

### 3.1 Route Facts

In [12]:
cabin_configs = (
    flight_capacity_by_class_2023.query("capacity > 0")
    .groupby("flight_number")["class_name"].unique().astype(str)
    .rename("cabin_configuration").reset_index()
)
cabin_conditions = (
    cabin_configs["cabin_configuration"].str.contains("First"),
    cabin_configs["cabin_configuration"].str.contains("Business"),
    cabin_configs["cabin_configuration"].str.contains("Economy")
)
cabin_choices = "(3) Economy, Business, First", "(2) Economy, Business", "(1) Economy only"
cabin_configs["cabin_configuration"] = np.select(cabin_conditions, cabin_choices, default="Unlisted config / Check logic")

flight_facts = (
    bookings_2023.groupby("flight_number").agg(
        revenue=("price_paid", "sum"),
        class_bookings=("booking_id", "count"),
        class_passengers=("checked_in", "sum")).reset_index()

    .merge(cabin_configs, on="flight_number")
    .merge(flights_2023[["flight_number", "flight_date", "line_number", "aircraft_id", "scheduled_arrival", "actual_arrival", "cancelled",
                         "cancellation_reason", "delay_reason_dep", "delay_reason_arr"]], on="flight_number")

    .merge(aircraft[["aircraft_id", "seat_capacity"]], on="aircraft_id")
    .merge(costs_per_flight_2023.drop(columns=["flight_cost_id"]), on=["flight_number", "flight_date"])
    .merge(routes_named[["line_number", "route_name_short"]], on="line_number")

).rename(columns={"seat_capacity": "nominal_seat_capacity"})

flight_facts["actual_seat_capacity"] = flight_facts["nominal_seat_capacity"].where(flight_facts["cancelled"] == False, np.nan)

flight_facts["revenue"] = flight_facts["revenue"].where(flight_facts["cancelled"] == False, np.nan)
for col in ["flight_cost_total", "fuel_cost", "crew_cost", "maintenance_cost", "landing_fees", "catering_cost", "other_costs"]:
    flight_facts[col] = flight_facts[col].where(flight_facts["cancelled"] == False, np.nan)

flight_facts["on_time"] = (flight_facts['actual_arrival'] - flight_facts['scheduled_arrival']).dt.total_seconds() <= (15 * 60)
flight_facts["delayed_departure"] = pd.notna(flight_facts["delay_reason_dep"])
flight_facts["delayed_arrival"] = pd.notna(flight_facts["delay_reason_arr"])

flight_facts["day_of_week"] = flight_facts["flight_date"].dt.day_name()

season_conditions = (
    flight_facts["flight_date"].dt.month.isin([1, 2]),
    flight_facts["flight_date"].dt.month.isin([3, 4, 5]),
    flight_facts["flight_date"].dt.month.isin([6, 7, 8]),
    flight_facts["flight_date"].dt.month.isin([9, 10, 11]),
    flight_facts["flight_date"].dt.month == 12
)
season_choices = ("(1) Jan & Feb", "(2) Spring", "(3) Summer", "(4) Autumn", "(5) December")
flight_facts["travel_season"] = np.select(season_conditions, season_choices, default="Unlisted month / Check logic")

weekday_conditions = (
    flight_facts["day_of_week"].isin(["Monday", "Wednesday", "Thursday"]),
    flight_facts["day_of_week"].isin(["Tuesday", "Saturday"]),
    flight_facts["day_of_week"].isin(["Friday", "Sunday"])
)
weekday_choices = ("(1) Mon/Wed/Thu", "(2) Tue/Sat", "(3) Fri/Sun")
flight_facts["weekday_group"] = np.select(weekday_conditions, weekday_choices, default="Unlisted weekday / Check logic")

route_facts = (
    flight_facts.groupby(["route_name_short", "cabin_configuration", "weekday_group", "travel_season"]).agg(
        flight_count=("flight_number", "count"),
        nominal_capacity_total=("nominal_seat_capacity", "sum"),
        actual_capacity_total=("actual_seat_capacity", "sum"),
        bookings_total=("class_bookings", "sum"),
        passengers_total=("class_passengers", "sum"),
        total_revenue=("revenue", "sum"),
        total_flight_cost_overall=("flight_cost_total", "sum"),
        total_fuel_cost=("fuel_cost", "sum"),
        total_crew_cost=("crew_cost", "sum"),
        total_maintenance_cost=("maintenance_cost", "sum"),
        total_landing_fees=("landing_fees", "sum"),
        total_catering_cost=("catering_cost", "sum"),
        total_other_costs=("other_costs", "sum"),
        cancellations_total=("cancelled", "sum"),
        departure_delays_total=("delayed_departure", "sum"),
        arrival_delays_total=("delayed_arrival", "sum"),
        on_time_arrivals_total=("on_time", "sum")
    )
).reset_index()

route_facts

,route_name_short,cabin_configuration,weekday_group,travel_season,flight_count,nominal_capacity_total,actual_capacity_total,bookings_total,passengers_total,total_revenue,...,total_fuel_cost,total_crew_cost,total_maintenance_cost,total_landing_fees,total_catering_cost,total_other_costs,cancellations_total,departure_delays_total,arrival_delays_total,on_time_arrivals_total
0,ALG → FRA,"(2) Economy, Business",(1) Mon/Wed/Thu,(1) Jan & Feb,50,9975,9.576,7368,6720,1.512.358,...,486.057,213.109,166.623,89.839,69.345,359.192,2,10,20,28
1,ALG → FRA,"(2) Economy, Business",(1) Mon/Wed/Thu,(2) Spring,80,15960,15.960,13058,12630,2.750.549,...,777.636,343.309,264.812,145.110,109.800,571.832,0,20,30,50
2,ALG → FRA,"(2) Economy, Business",(1) Mon/Wed/Thu,(3) Summer,80,15960,15.767,14414,13888,2.977.522,...,754.239,334.664,262.142,137.209,107.248,550.657,1,16,27,52
3,ALG → FRA,"(2) Economy, Business",(1) Mon/Wed/Thu,(4) Autumn,78,15561,15.355,12320,11687,2.584.573,...,763.911,343.269,262.881,143.856,110.771,568.200,1,16,22,55
4,ALG → FRA,"(2) Economy, Business",(1) Mon/Wed/Thu,(5) December,24,4788,4.788,4361,4278,909.510,...,230.616,102.337,80.298,42.598,32.796,169.008,0,4,10,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2530,ZRH → FRA,"(2) Economy, Business",(3) Fri/Sun,(1) Jan & Feb,34,6542,6.542,4670,4395,222.230,...,72.049,32.224,24.824,13.060,10.485,54.092,0,9,9,25
2531,ZRH → FRA,"(2) Economy, Business",(3) Fri/Sun,(2) Spring,52,9866,9.866,7758,7499,364.401,...,100.924,44.958,35.273,19.031,14.081,73.595,0,12,17,35
2532,ZRH → FRA,"(2) Economy, Business",(3) Fri/Sun,(3) Summer,52,10024,9.813,8611,8166,388.888,...,95.817,42.448,33.472,17.553,14.123,68.150,1,12,17,34
2533,ZRH → FRA,"(2) Economy, Business",(3) Fri/Sun,(4) Autumn,52,9866,8.890,7520,6466,317.985,...,91.994,40.512,31.609,17.217,13.150,70.832,5,11,17,30


In [ ]:
route_tiers = (
    flight_facts.groupby("route_name_short").agg(
    nominal_capacity_total=("nominal_seat_capacity", "sum"),
    actual_capacity_total=("actual_seat_capacity", "sum"),
    bookings_total=("class_bookings", "sum"),
    total_revenue=("revenue", "sum"),
    total_flight_cost_overall=("flight_cost_total", "sum")
    ).reset_index()
)

route_tiers["avg_booked_rate_pct"] = route_tiers["bookings_total"] / route_tiers["nominal_capacity_total"] * 100
route_tiers["total_profit"] = route_tiers["total_revenue"] - route_tiers["total_flight_cost_overall"]
route_tiers["avg_profit_margin_pct"] = route_tiers["total_profit"] / route_tiers["total_revenue"] * 100

booking_tier_conditions = (
    route_tiers["avg_booked_rate_pct"] > 85,
    route_tiers["avg_booked_rate_pct"] > 75,
    route_tiers["avg_booked_rate_pct"] >= 70,
    route_tiers["avg_booked_rate_pct"] >= 60,
    route_tiers["avg_booked_rate_pct"] < 60
)
booking_tier_choices = (
    "(A) Top Performance (> 85% Avg Booked Rate)",
    "(B) Within Target (> 76–85% Avg Booked Rate)",
    "(C) Sufficient (> 70–75% Avg Booked Rate)",
    "(D) Underperforming (< 70% Avg Booked Rate)",
    "(E) Unsustainable (< 60% Avg Booked Rate)"
)
route_tiers["booked_performance_tier"] = np.select(booking_tier_conditions, booking_tier_choices, default="Unclassified / Check Logic")

profit_tier_conditions = (
    route_tiers["avg_profit_margin_pct"] >= 25,
    route_tiers["avg_profit_margin_pct"] >= 20,
    route_tiers["avg_profit_margin_pct"] >= 15,
    route_tiers["avg_profit_margin_pct"] >= 5,
    route_tiers["avg_profit_margin_pct"] < 5,
)
profit_tier_choices = (
    "(A) Top Profit Margin (>= 25%)",
    "(B) High Profit Margin (20–24%)",
    "(C) Healthy Profit Margin (15–19%)",
    "(D) Small Profit Margin (5–14%)",
    "(E) Loss Risk (Profit Margin < 5%)"
)
route_tiers["profitability_tier"] = np.select(profit_tier_conditions, profit_tier_choices, default="Unclassified / Check Logic")

#route_tiers

In [14]:
route_facts = route_facts.merge(route_tiers[["route_name_short", "booked_performance_tier", "profitability_tier"]], on="route_name_short")
route_facts

,route_name_short,cabin_configuration,weekday_group,travel_season,flight_count,nominal_capacity_total,actual_capacity_total,bookings_total,passengers_total,total_revenue,...,total_maintenance_cost,total_landing_fees,total_catering_cost,total_other_costs,cancellations_total,departure_delays_total,arrival_delays_total,on_time_arrivals_total,booked_performance_tier,profitability_tier
0,ALG → FRA,"(2) Economy, Business",(1) Mon/Wed/Thu,(1) Jan & Feb,50,9975,9.576,7368,6720,1.512.358,...,166.623,89.839,69.345,359.192,2,10,20,28,(B) Within Target (> 76-85% Avg Booked Rate),(C) Healthy Profit Margin (15-19%)
1,ALG → FRA,"(2) Economy, Business",(1) Mon/Wed/Thu,(2) Spring,80,15960,15.960,13058,12630,2.750.549,...,264.812,145.110,109.800,571.832,0,20,30,50,(B) Within Target (> 76-85% Avg Booked Rate),(C) Healthy Profit Margin (15-19%)
2,ALG → FRA,"(2) Economy, Business",(1) Mon/Wed/Thu,(3) Summer,80,15960,15.767,14414,13888,2.977.522,...,262.142,137.209,107.248,550.657,1,16,27,52,(B) Within Target (> 76-85% Avg Booked Rate),(C) Healthy Profit Margin (15-19%)
3,ALG → FRA,"(2) Economy, Business",(1) Mon/Wed/Thu,(4) Autumn,78,15561,15.355,12320,11687,2.584.573,...,262.881,143.856,110.771,568.200,1,16,22,55,(B) Within Target (> 76-85% Avg Booked Rate),(C) Healthy Profit Margin (15-19%)
4,ALG → FRA,"(2) Economy, Business",(1) Mon/Wed/Thu,(5) December,24,4788,4.788,4361,4278,909.510,...,80.298,42.598,32.796,169.008,0,4,10,14,(B) Within Target (> 76-85% Avg Booked Rate),(C) Healthy Profit Margin (15-19%)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2530,ZRH → FRA,"(2) Economy, Business",(3) Fri/Sun,(1) Jan & Feb,34,6542,6.542,4670,4395,222.230,...,24.824,13.060,10.485,54.092,0,9,9,25,(C) Sufficient (> 70-75% Avg Booked Rate),(D) Small Profit Margin (5-14%)
2531,ZRH → FRA,"(2) Economy, Business",(3) Fri/Sun,(2) Spring,52,9866,9.866,7758,7499,364.401,...,35.273,19.031,14.081,73.595,0,12,17,35,(C) Sufficient (> 70-75% Avg Booked Rate),(D) Small Profit Margin (5-14%)
2532,ZRH → FRA,"(2) Economy, Business",(3) Fri/Sun,(3) Summer,52,10024,9.813,8611,8166,388.888,...,33.472,17.553,14.123,68.150,1,12,17,34,(C) Sufficient (> 70-75% Avg Booked Rate),(D) Small Profit Margin (5-14%)
2533,ZRH → FRA,"(2) Economy, Business",(3) Fri/Sun,(4) Autumn,52,9866,8.890,7520,6466,317.985,...,31.609,17.217,13.150,70.832,5,11,17,30,(C) Sufficient (> 70-75% Avg Booked Rate),(D) Small Profit Margin (5-14%)


In [15]:
route_facts.to_csv(csv_folder_path + "route_facts.csv", index=False, sep=";", decimal=",")

### 3.2 Aircraft Usage

In [16]:
flights_aircraft = (
    flights_2023[["flight_number", "flight_date", "line_number", "aircraft_id"]]
    .merge(routes_named[["line_number", "route_name_short"]], on="line_number")
    .merge(aircraft[["aircraft_id", "manufacturer", "model", "seat_capacity", "range_km"]], on="aircraft_id")
)

season_conditions = (
    flights_aircraft["flight_date"].dt.month.isin([1, 2]),
    flights_aircraft["flight_date"].dt.month.isin([3, 4, 5]),
    flights_aircraft["flight_date"].dt.month.isin([6, 7, 8]),
    flights_aircraft["flight_date"].dt.month.isin([9, 10, 11]),
    flights_aircraft["flight_date"].dt.month == 12
)
flights_aircraft["travel_season"] = np.select(season_conditions, season_choices, default="Unlisted month / Check logic")

flights_aircraft["day_of_week"] = flights_aircraft["flight_date"].dt.day_name()

weekday_conditions = (
    flights_aircraft["day_of_week"].isin(["Monday", "Wednesday", "Thursday"]),
    flights_aircraft["day_of_week"].isin(["Tuesday", "Saturday"]),
    flights_aircraft["day_of_week"].isin(["Friday", "Sunday"])
)
flights_aircraft["weekday_group"] = np.select(weekday_conditions, weekday_choices, default="Unlisted day / Check logic")

aircraft_usage = (
    flights_aircraft.groupby(["route_name_short", "weekday_group", "travel_season", "manufacturer", "model", "seat_capacity", "range_km"]).agg(
        flight_count=("flight_number", "count")
    ).reset_index()
)

aircraft_usage

,route_name_short,weekday_group,travel_season,manufacturer,model,range_km,flight_count
0,ALG → FRA,(1) Mon/Wed/Thu,(1) Jan & Feb,Airbus,A321neo,7400,25
1,ALG → FRA,(1) Mon/Wed/Thu,(1) Jan & Feb,Boeing,737 MAX 9,6575,25
2,ALG → FRA,(1) Mon/Wed/Thu,(2) Spring,Airbus,A321neo,7400,40
3,ALG → FRA,(1) Mon/Wed/Thu,(2) Spring,Boeing,737 MAX 9,6575,40
4,ALG → FRA,(1) Mon/Wed/Thu,(3) Summer,Airbus,A321neo,7400,40
...,...,...,...,...,...,...,...
4232,ZRH → FRA,(3) Fri/Sun,(3) Summer,Irkut,MC-21-300,6000,40
4233,ZRH → FRA,(3) Fri/Sun,(4) Autumn,Embraer,E195-E2,4815,14
4234,ZRH → FRA,(3) Fri/Sun,(4) Autumn,Irkut,MC-21-300,6000,38
4235,ZRH → FRA,(3) Fri/Sun,(5) December,Embraer,E195-E2,4815,4


In [17]:
aircraft_usage.to_csv(csv_folder_path + "aircraft_usage.csv", index=False, sep=";", decimal=",")

### 3.3 Delays and Cancellations

In [20]:
delayed_flights = flights_2023.copy().merge(routes_named[["line_number", "route_name_short"]], on="line_number")

delayed_flights["delay_minutes"] = (delayed_flights["actual_arrival"] - delayed_flights["scheduled_arrival"]).dt.total_seconds() / 60
delayed_flights["delay_minutes"] = delayed_flights["delay_minutes"].where(delayed_flights["delay_minutes"] > 15, np.nan)

season_conditions = (
    delayed_flights["flight_date"].dt.month.isin([1, 2]),
    delayed_flights["flight_date"].dt.month.isin([3, 4, 5]),
    delayed_flights["flight_date"].dt.month.isin([6, 7, 8]),
    delayed_flights["flight_date"].dt.month.isin([9, 10, 11]),
    delayed_flights["flight_date"].dt.month == 12
)
delayed_flights["travel_season"] = np.select(season_conditions, season_choices, default="Unlisted month / Check logic")

delayed_flights["day_of_week"] = delayed_flights["flight_date"].dt.day_name()
weekday_conditions = (
    delayed_flights["day_of_week"].isin(["Monday", "Wednesday", "Thursday"]),
    delayed_flights["day_of_week"].isin(["Tuesday", "Saturday"]),
    delayed_flights["day_of_week"].isin(["Friday", "Sunday"])
)
delayed_flights["weekday_group"] = np.select(weekday_conditions, weekday_choices, default="Unlisted day / Check logic")

mask_dep_missing = (
    delayed_flights["delay_reason_dep"].isna()
    & delayed_flights["delay_reason_arr"].notna()
)
mask_arr_missing = (
    delayed_flights["delay_reason_arr"].isna()
    & delayed_flights["delay_reason_dep"].notna()
)
delayed_flights.loc[mask_dep_missing, "delay_reason_dep"] = "No departure delay"
delayed_flights.loc[mask_arr_missing, "delay_reason_arr"] = "No arrival delay"

flight_delays = (
    delayed_flights.groupby(["route_name_short", "delay_reason_dep", "delay_reason_arr", "weekday_group", "travel_season"]).agg(
        flight_count=("flight_number", "count"),
        avg_delay_minutes=("delay_minutes", "mean"),
        max_delay_minutes=("delay_minutes", "max")
    ).reset_index()
)

flight_delays["delay_reason_dep"] = flight_delays["delay_reason_dep"].where(flight_delays["delay_reason_dep"] != "No departure delay", pd.NA)
flight_delays["delay_reason_arr"] = flight_delays["delay_reason_arr"].where(flight_delays["delay_reason_arr"] != "No arrival delay", pd.NA)

flight_delays

,route_name_short,delay_reason_dep,delay_reason_arr,weekday_group,travel_season,flight_count,avg_delay_minutes,max_delay_minutes
0,ALG → FRA,Air traffic control,Late departure,(1) Mon/Wed/Thu,(1) Jan & Feb,2,"25,00","28,00"
1,ALG → FRA,Air traffic control,Late departure,(1) Mon/Wed/Thu,(2) Spring,2,"25,00","32,00"
2,ALG → FRA,Air traffic control,Late departure,(1) Mon/Wed/Thu,(3) Summer,4,"28,75","45,00"
3,ALG → FRA,Air traffic control,Late departure,(1) Mon/Wed/Thu,(4) Autumn,4,"30,50","45,00"
4,ALG → FRA,Air traffic control,Late departure,(1) Mon/Wed/Thu,(5) December,2,"26,50","33,00"
...,...,...,...,...,...,...,...,...
13243,ZRH → FRA,Weather,Late departure,(2) Tue/Sat,(4) Autumn,3,"66,67","93,00"
13244,ZRH → FRA,Weather,Late departure,(3) Fri/Sun,(1) Jan & Feb,1,"45,00","45,00"
13245,ZRH → FRA,Weather,Late departure,(3) Fri/Sun,(2) Spring,2,"48,50","52,00"
13246,ZRH → FRA,Weather,Late departure,(3) Fri/Sun,(4) Autumn,2,"43,50","50,00"


In [21]:
flight_delays.to_csv(csv_folder_path + "flight_delays.csv", index=False, sep=";", decimal=",")

In [ ]:
canceled_flights = (
    flights_2023.query("cancelled == True")[["flight_number", "flight_date", "line_number", "cancellation_reason"]]
    .merge(routes_named[["line_number", "route_name_short"]], on="line_number")
    .drop(columns=["line_number"])
)

season_conditions = (
    canceled_flights["flight_date"].dt.month.isin([1, 2]),
    canceled_flights["flight_date"].dt.month.isin([3, 4, 5]),
    canceled_flights["flight_date"].dt.month.isin([6, 7, 8]),
    canceled_flights["flight_date"].dt.month.isin([9, 10, 11]),
    canceled_flights["flight_date"].dt.month == 12
)
canceled_flights["travel_season"] = np.select(season_conditions, season_choices, default="Unlisted month / Check logic")

canceled_flights["day_of_week"] = canceled_flights["flight_date"].dt.day_name()
weekday_conditions = (
    canceled_flights["day_of_week"].isin(["Monday", "Wednesday", "Thursday"]),
    canceled_flights["day_of_week"].isin(["Tuesday", "Saturday"]),
    canceled_flights["day_of_week"].isin(["Friday", "Sunday"])
)
canceled_flights["weekday_group"] = np.select(weekday_conditions, weekday_choices, default="Unlisted day / Check logic")

flight_cancellations = (
    canceled_flights.groupby(["route_name_short", "weekday_group", "travel_season", "cancellation_reason"]).agg(
        cancellation_count=("flight_number", "count")
    ).reset_index()
)

flight_cancellations

In [ ]:
flight_cancellations.to_csv(csv_folder_path + "flight_cancellations.csv", index=False, sep=";", decimal=",")

## 4. Weather Hazards

In [ ]:
airport_weather = (
    weather_2023[
        ["airport_code", "season", "temperature_celsius", "wind_speed_kmh", "visibility_km", "condition_description"]]
    .merge(airport_names_clean[["airport_code", "airport_name", "coordinates", "climate_region"]], on="airport_code")
)

airport_weather["fog"] = (airport_weather["condition_description"] == "Fog")
airport_weather["blizzard"] = (airport_weather["condition_description"] == "Blizzard")
airport_weather["sandstorm"] = (airport_weather["condition_description"] == "Sandstorm")
airport_weather["extreme_cold"] = (airport_weather["temperature_celsius"] < -25)
airport_weather["extreme_heat"] = (airport_weather["temperature_celsius"] > 45)
airport_weather["wind_speed_above_70_kmh"] = (airport_weather["wind_speed_kmh"] > 70)
airport_weather["visibility_under_1_km"] = (airport_weather["visibility_km"] < 1)

weather_hazards = (
    airport_weather.groupby(["climate_region", "season", "airport_name", "coordinates"])
        [["fog", "blizzard", "sandstorm", "extreme_cold", "extreme_heat",
          "wind_speed_above_70_kmh", "visibility_under_1_km",]].sum().reset_index()
)

weather_hazards = weather_hazards.melt(
    id_vars=["climate_region", "season", "airport_name", "coordinates"],
    var_name="hazard_type",
    value_name="hazard_count"
)

rename_map = {
    "fog": "Fog",
    "blizzard": "Blizzard",
    "sandstorm": "Sandstorm",
    "extreme_cold": "Extreme Cold",
    "extreme_heat": "Extreme Heat",
    "wind_speed_above_70_kmh": "Strong Wind",
    "visibility_under_1_km": "Low Visibility"
}

weather_hazards["hazard_type"] = weather_hazards["hazard_type"].map(rename_map)

def split_coordinates(coordinates):
    coordinates = coordinates.replace("(", "").replace(")", "")
    longitude, latitude = coordinates.split(",")
    return float(longitude), float(latitude)

weather_hazards["longitude"], weather_hazards["latitude"] = zip(*weather_hazards["coordinates"].apply(split_coordinates))
weather_hazards.drop(columns=["coordinates"], inplace=True)

weather_hazards

In [ ]:
weather_hazards.to_csv(csv_folder_path + "weather_hazards.csv", index=False, sep=";", decimal=",")

## 5. Customers

In [25]:
customers_canonical = customers_2023.copy()

earliest_flights = bookings_2023.groupby("customer_id")["flight_date"].min().rename("earliest_flight_date").reset_index()
customers_canonical = customers_canonical.merge(earliest_flights, on="customer_id")

customers_canonical["date_of_birth"] = pd.to_datetime(customers_canonical["date_of_birth"])
years = customers_canonical["earliest_flight_date"].dt.year - customers_canonical["date_of_birth"].dt.year
had_birthday = (
    customers_canonical["earliest_flight_date"].dt.month > customers_canonical["date_of_birth"].dt.month) | (
    (customers_canonical["earliest_flight_date"].dt.month == customers_canonical["date_of_birth"].dt.month) &
    (customers_canonical["earliest_flight_date"].dt.day >= customers_canonical["date_of_birth"].dt.day)
)
customers_canonical["canonical_age"] = np.where(
    customers_canonical["date_of_birth"].isna() | customers_canonical["earliest_flight_date"].isna(),
    pd.NA, years - (~had_birthday).astype(int)
)

age_conditions = (
    customers_canonical["canonical_age"] <= 24,
    customers_canonical["canonical_age"] <= 34,
    customers_canonical["canonical_age"] <= 44,
    customers_canonical["canonical_age"] <= 60,
    customers_canonical["canonical_age"] > 60
)
age_choices = ("(1) Age <= 24", "(2) Age 25-34", "(3) Age 35-44", "(4) Age 45-60", "(5) Age > 60")
customers_canonical["age_group"] = np.select(age_conditions, age_choices, default="(6) Age unknown")

customer_attributes = (customers_canonical
                      .groupby(["frequent_flyer_status_code", "nationality", "gender", "age_group"])
                      .agg(distinct_customers=("customer_id", "nunique"))
).reset_index()

customer_attributes

,frequent_flyer_status_code,nationality,gender,age_group,distinct_customers
0,0-N,Australia,female,(1) Age <= 24,5372
1,0-N,Australia,female,(2) Age 25-34,5842
2,0-N,Australia,female,(3) Age 35-44,1798
3,0-N,Australia,female,(4) Age 45-60,337
4,0-N,Australia,female,(5) Age > 60,350
...,...,...,...,...,...
1050,4-P,United States,male,(2) Age 25-34,12
1051,4-P,United States,male,(3) Age 35-44,14
1052,4-P,United States,male,(4) Age 45-60,20
1053,4-P,United States,male,(5) Age > 60,32


In [26]:
customer_attributes.to_csv(csv_folder_path + "customer_attributes.csv", index=False, sep=";", decimal=",")

In [28]:
customers_bookings = (
    bookings_2023[["booking_id", "customer_id", "flight_date", "booking_time", "class_name", "frequent_flyer_status_code", "checked_in"]]
    .merge(customers_2023[["customer_id", "date_of_birth", "nationality", "gender"]], on="customer_id")
)

customers_bookings["booking_date"] = pd.to_datetime(customers_bookings["booking_time"].dt.date)
customers_bookings["booking_time"] = customers_bookings["booking_time"].dt.time

customers_bookings["booking_lead_time_days"] = (customers_bookings["flight_date"] - customers_bookings["booking_date"]).dt.days

booking_time_conditions = (
    customers_bookings["booking_time"].between(pd.to_datetime("06:00:00").time(), pd.to_datetime("12:00:00").time()),
    customers_bookings["booking_time"].between(pd.to_datetime("12:00:00").time(), pd.to_datetime("17:00:00").time()),
    customers_bookings["booking_time"].between(pd.to_datetime("17:00:00").time(), pd.to_datetime("22:00:00").time()),
    (customers_bookings["booking_time"] >= pd.to_datetime("22:00:00").time()) | (customers_bookings["booking_time"] < pd.to_datetime("06:00:00").time())
)
booking_time_choices = ("06:00 - 12:00", "12:00 - 17:00", "17:00 - 22:00", "22:00 - 06:00")
customers_bookings["booking_time_of_day"] = np.select(booking_time_conditions, booking_time_choices, default="Unlisted time of day / Check logic")

lead_time_conditions = (
    customers_bookings["booking_lead_time_days"] <= 28,
    customers_bookings["booking_lead_time_days"] <= 42,
    customers_bookings["booking_lead_time_days"] <= 56,
    customers_bookings["booking_lead_time_days"] <= 70,
    customers_bookings["booking_lead_time_days"] <= 84,
    customers_bookings["booking_lead_time_days"] > 84
)
lead_time_choices = (
    "(1) Within 4 week window", "(2) Within 6 week window", "(3) Within 8 week window",
    "(4) Within 10 week window", "(5) Within 12 week window", "(6) More than 12 weeks"
)
customers_bookings["booking_lead_time"] = np.select(lead_time_conditions, lead_time_choices, default="Unlisted lead time / Check logic")

customers_bookings["date_of_birth"] = pd.to_datetime(customers_bookings["date_of_birth"])
years = customers_bookings["flight_date"].dt.year - customers_bookings["date_of_birth"].dt.year
had_birthday = (
    customers_bookings["flight_date"].dt.month > customers_bookings["date_of_birth"].dt.month) | (
    (customers_bookings["flight_date"].dt.month == customers_bookings["date_of_birth"].dt.month) &
    (customers_bookings["flight_date"].dt.day >= customers_bookings["date_of_birth"].dt.day)
)
customers_bookings["booking_age"] = np.where(
    customers_bookings["date_of_birth"].isna() | customers_bookings["flight_date"].isna(),
    pd.NA, years - (~had_birthday).astype(int)
)

age_conditions = (
    customers_bookings["booking_age"] <= 24,
    customers_bookings["booking_age"] <= 34,
    customers_bookings["booking_age"] <= 44,
    customers_bookings["booking_age"] <= 60,
    customers_bookings["booking_age"] > 60
)
customers_bookings["age_group"] = np.select(age_conditions, age_choices, default="(6) Age unknown")

customer_behavior = (customers_bookings
                   .groupby(["class_name", "frequent_flyer_status_code", "nationality", "gender", "age_group", "booking_time_of_day", "booking_lead_time"])
                   .agg(bookings=("booking_id", "count"), check_ins=("checked_in", "sum"))
).reset_index()

customer_behavior

,class_name,frequent_flyer_status_code,nationality,gender,age_group,booking_time_of_day,booking_lead_time,bookings,check_ins
0,Business,0-N,Australia,female,(1) Age <= 24,06:00 - 12:00,(1) Within 4 week window,10,8
1,Business,0-N,Australia,female,(1) Age <= 24,06:00 - 12:00,(2) Within 6 week window,39,36
2,Business,0-N,Australia,female,(1) Age <= 24,06:00 - 12:00,(3) Within 8 week window,70,68
3,Business,0-N,Australia,female,(1) Age <= 24,06:00 - 12:00,(4) Within 10 week window,17,15
4,Business,0-N,Australia,female,(1) Age <= 24,06:00 - 12:00,(5) Within 12 week window,1,1
...,...,...,...,...,...,...,...,...,...
44124,First,4-P,United States,female,(2) Age 25-34,17:00 - 22:00,(3) Within 8 week window,1,1
44125,First,4-P,United States,female,(4) Age 45-60,17:00 - 22:00,(4) Within 10 week window,1,1
44126,First,4-P,United States,male,(1) Age <= 24,12:00 - 17:00,(4) Within 10 week window,1,1
44127,First,4-P,United States,male,(4) Age 45-60,06:00 - 12:00,(3) Within 8 week window,1,1


In [28]:
customer_behavior.to_csv(csv_folder_path + "customer_behavior.csv", index=False, sep=";", decimal=",")